In [1]:
from __future__ import absolute_import, division, print_function
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt 
import numpy as np
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

In [2]:
def logsumexp(vec):
    max_score, _ = vec.max(dim=0) 
    lse = max_score + torch.log(torch.sum(torch.exp(vec - max_score)))
    return lse

def loss_plain_softmax(x1,index):
    return torch.log(torch.exp(x1[index])/(torch.exp(x1).sum()))

def loss_logsumexp(x1,index):
    return x1[index]-torch.log(torch.exp(x1).sum())

def loss_logsumexp_stable(x1,index):
    return x1[index] -torch.logsumexp(x1,0)

def loss_logsumexp_stable_own(x1,index):
    return x1[index] -logsumexp(x1)

def compute_crossentropyloss_manual(func,x,y0):
    """
    x is the vector with shape (batch_size,C)
    y0 shape is the same (batch_size), whose entries are integers from 0 to C-1
    C are the classes
    func is the name of the function, e.g., loss_logsumexp that you will call
    """
    loss = 0.
    n_batch, n_class = x.shape
    for x1,y1 in zip(x,y0):        
        
        loss = loss + func(x1,y1)
    loss = - loss/n_batch
    return loss

def compute_percentage_loss(loss_name,uni_s,uni_e):
    #torch.manual_seed(0)
    precision = 3
    batch_size=10
    C = 15
    N_iter = 100
    n_correct_CE = 0
    criterion2 = nn.CrossEntropyLoss()
    for i in range(N_iter):
        x = torch.rand(size=(batch_size,C)).uniform_(uni_s,uni_e).to(torch.float)   
        y0 = torch.randint(0,C,size=(batch_size,))
        CEloss = criterion2(x,y0)
        manual_CEloss = compute_crossentropyloss_manual(loss_name,x,y0)
        if i==0:
            print('CrossEntropyLoss:')
            print('module:%s'%(str(CEloss)))
            print('manual:%s'%(str(manual_CEloss)))


        CE_loss_check = np.abs((CEloss- manual_CEloss).item())<10**-precision
        if CE_loss_check: n_correct_CE+=1

    print('percentage CELoss correctly computed :%s'%(str(n_correct_CE/N_iter*100)))

In [3]:
# Compute the percentage loss when the uniform distribution takes values 
#between 0 and 1 
start=0
end=1
compute_percentage_loss(loss_plain_softmax,start,end)
compute_percentage_loss(loss_logsumexp,start,end)
compute_percentage_loss(loss_logsumexp_stable,start,end)
compute_percentage_loss(loss_logsumexp_stable_own,start,end)

CrossEntropyLoss:
module:tensor(2.6885)
manual:tensor(2.6885)
percentage CELoss correctly computed :100.0
CrossEntropyLoss:
module:tensor(2.7132)
manual:tensor(2.7132)
percentage CELoss correctly computed :100.0
CrossEntropyLoss:
module:tensor(2.7239)
manual:tensor(2.7239)
percentage CELoss correctly computed :100.0
CrossEntropyLoss:
module:tensor(2.6372)
manual:tensor(2.6372)
percentage CELoss correctly computed :100.0


In [4]:
# Compute the percentage loss when the uniform distribution takes values 
#between 0 and 100 
start=0
end=100
compute_percentage_loss(loss_plain_softmax,start,end)
compute_percentage_loss(loss_logsumexp,start,end)
compute_percentage_loss(loss_logsumexp_stable,start,end)
compute_percentage_loss(loss_logsumexp_stable_own,start,end)

CrossEntropyLoss:
module:tensor(40.1088)
manual:tensor(inf)
percentage CELoss correctly computed :0.0
CrossEntropyLoss:
module:tensor(53.3572)
manual:tensor(inf)
percentage CELoss correctly computed :0.0
CrossEntropyLoss:
module:tensor(53.9245)
manual:tensor(53.9245)
percentage CELoss correctly computed :100.0
CrossEntropyLoss:
module:tensor(47.3778)
manual:tensor(47.3778)
percentage CELoss correctly computed :100.0
